In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
import seaborn as sns
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,plot_confusion_matrix,classification_report
import time

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
device

In [ ]:
train=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train.info()
train

In [ ]:
test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test.info()
test

In [ ]:
data_train=train['text'].values
label=train['target'].values
x_train,x_val,label_train,label_val=train_test_split(data_train,label,test_size=0.10,random_state=42,shuffle=True)

In [ ]:
print(len(x_train))
print(len(x_val))



In [ ]:
sns.countplot(label);

In [ ]:
import re
def preprocessing(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)
    
    return s

In [ ]:
from nltk.corpus import stopwords

def tokenizer(x_train,x_val):
    word_list=[]
    stop_word=set(stopwords.words('english'))
    for sent in x_train:
        sent=sent.lower().split()
        for word in sent:
            word=preprocessing(word)
           # if word not in stop_word and word != '' : 
            word_list.append(word)
                
                
    corpus=Counter(word_list)     
    corpus_=sorted(corpus,key=corpus.get,reverse=True)[:1500]
    one_hot_dict={w:i+1 for i,w in enumerate(corpus_)}
    
    train_list=[]
    val_list=[] 
    for  sent in  x_train:   
         train_list.append( [one_hot_dict[preprocessing(word)] for word in sent.lower().split()
                           if preprocessing(word) in one_hot_dict.keys()])
        
    for sent in x_val :
        val_list.append( [one_hot_dict[preprocessing(word)]  for word in sent.lower().split()
                            if preprocessing(word) in one_hot_dict.keys()])
    
    
    
    return  train_list,val_list,one_hot_dict
        
x_tr,x_ts,vocab=tokenizer(x_train,x_val)

In [ ]:
#Determine max length from expressions
a=[len(x) for x in x_tr]
max_len_index=a.index(max(a))
max_len_seq=len(x_tr[max_len_index])
max_len_seq

In [ ]:
#padding used to same size tensors
pad=max_len_seq
def padding(data,padd):
    for i,sent in enumerate(data):
        feature=np.zeros((1,padd),dtype=int) 
        if len(np.array(sent)) != 0:
            feature[:,-len(sent):]=np.array(sent)
            data[i]=feature
        else :
            data[i]=feature
    return data

X_train=padding(x_tr,pad) 
X_val=padding(x_ts,pad)

In [ ]:
class sentimentLSTM(nn.Module):
    def __init__(self,num_layers,batch_size,hidden_size,vocab_size,embed_size,p,pad):
        super(sentimentLSTM,self).__init__()
        
        self.hidden=hidden_size
        self.embed=nn.Embedding(vocab_size,embed_size)
        self.embed_size=embed_size
        self.num_layers=num_layers
        self.p=p
        self.pad=pad
        self.lstm=nn.LSTM(input_size=self.embed_size,
                         hidden_size=self.hidden,
                         num_layers=self.num_layers,
                         batch_first=True)
        

        
        
        self.linear=nn.Linear(self.hidden,512)
        self.linear2=nn.Linear(512,1)

        self.drop=nn.Dropout(self.p)
        self.relu=nn.ReLU()
      #  self.sigmoid=nn.Sigmoid()
    

    
    def forward(self,x,hidden):
        
        batch=x.shape[0]
        x=x.view(batch,-1)
        x=self.embed(x)
        x,hidden=self.lstm(x)
        self.x=x.contiguous().view(-1, self.hidden)
        x=self.drop(self.x)
        x=self.relu(x)
        x=self.linear(x) 
        
        x=self.drop(x)
        x=self.relu(x)
        x=self.linear2(x)  
        
        
       # x=self.sigmoid(x)
        out=x.view(batch,self.pad, -1)   
        
        return out[:,-1] ,h
    
    
    def get_x(self):
        return self.x

    def init_hidden(self,b):

        h0 = torch.zeros((self.num_layers,batch_size,self.hidden)).to(device)
        c0 = torch.zeros((self.num_layers,batch_size,self.hidden)).to(device)
        hidden = (h0,c0)
        return hidden

In [ ]:
epochs=20
lr=0.01
hidden_size=512
vocab_size=len(vocab)+1
embed_size=250
batch_size=64
num_layers=1
p=0.5
model=sentimentLSTM(num_layers,batch_size,hidden_size,vocab_size,embed_size,p,pad).to(device)
criterion=nn.BCEWithLogitsLoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

train_data=TensorDataset(torch.tensor(X_train),torch.tensor(label_train))
val_data=TensorDataset(torch.tensor(X_val),torch.tensor(label_val))

train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_data,batch_size=batch_size,shuffle=True)



train_losses=[]
val_losses=[]
for epoch in range(epochs):
    
    train_loss=0
    val_loss=0
    model.train()
    train_preds=[]
    train_labels=[]
    h = model.init_hidden(batch_size)
    for e,(train_inputs,train_label) in enumerate(train_loader):
        
        train_inputs=train_inputs.type(torch.LongTensor).to(device)
        train_label=train_label.type(torch.LongTensor).to(device)
        
        h = tuple([each.data for each in h])
        train_pred,h=model(train_inputs,h)

        loss=criterion(train_pred.float(),train_label.reshape(-1,1).float())
        train_loss+=loss.item()
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_labels.append(np.array(train_label.cpu()))
        train_preds.append(np.array([1 if i>0.5 else 0 for i in train_pred]))
        
    train_losses.append(train_loss/len(train_data)) 
    train_labels=np.concatenate(train_labels)
    train_preds=np.concatenate(train_preds)
    acc_train=accuracy_score(train_labels,train_preds)    

    model.eval() 
    val_preds=[]
    val_labels=[]
    for t,(input_val,l_val) in enumerate(val_loader):
        
        input_val=input_val.type(torch.LongTensor).to(device)
        l_val=l_val.type(torch.LongTensor).to(device)
        
        h = tuple([each.data for each in h])
        pred_val,h=model(input_val,h)

        loss_val=criterion(pred_val.float(),l_val.reshape(-1,1).float())
        val_loss+=loss_val.item()
        
        
        val_labels.append(np.array(l_val.cpu()))
        val_preds.append(np.array([1 if i>0.5 else 0 for i in pred_val ]))
        
    
    val_losses.append(val_loss/len(val_data))
    val_labels=np.concatenate(val_labels)
    val_preds=np.concatenate(val_preds)
    acc_val=accuracy_score(val_labels,val_preds)
    
    print('Epochs {}, Train_accurary {:.3f},Train error {:.5f} --- Val_accuracy {:.3f}'.
          format(epoch,acc_train,(train_loss/len(train_data)),acc_val))

In [ ]:
test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_text=test['text']
test_text

In [ ]:

#test text preprocessing 
test_list=[]
for text in test_text:
    word_seq = np.array([vocab[preprocessing(word)] for word in text.lower().split() 
                     if preprocessing(word)  in vocab.keys()])
    feature=np.zeros((1,max_len_seq),dtype=int)

    if len(word_seq)==0 :
        inputs =  torch.from_numpy(feature).type(torch.LongTensor).to(device)
        test_list.append(inputs)

    else:
        feature[:,-len(word_seq):]=np.array(word_seq)
        inputs =  torch.from_numpy(feature).type(torch.LongTensor).to(device)
        test_list.append(inputs)


In [ ]:
outputs=[]
for seq in test_list:
    model.eval()
    with torch.no_grad():
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output, h = model(seq, h)
        outputs.append(1 if output>0.5 else 0  )
        

In [ ]:
submission=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')


In [ ]:
acc_scr=accuracy_score(np.array(outputs),np.array(submission['target']))
acc_scr

In [ ]:
f1_score(np.array(outputs),np.array(submission['target']))

In [ ]:
submission_new=pd.DataFrame()
submission_new['target']=np.array(outputs)
submission_new.index=submission['id']
submission_new.to_csv('/kaggle/working/submission.csv')
submission_new

In [ ]:
print(classification_report(outputs,list(submission['target'])))

In [ ]:
sns.countplot(submission['target'])

In [ ]:
sns.countplot(np.array(outputs))